<a href="https://colab.research.google.com/github/sruthi-v04/NER-using-gliner-and-uniner/blob/main/UNIneripynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentencepiece

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,

)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: {input_text}
              ASSISTANT: I’ve read this text.
              USER: What describes {entity_type} in the text?
              ASSISTANT:
           """

prompt = PromptTemplate(template=template, input_variables=["input_text","entity_type"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
text_4= """ In a traditional community, Sarah, a talented young woman, dreams of pursuing her passion for engineering. However, she faces resistance from her family and community, who believe that women should prioritize marriage and domestic duties over education and career aspirations. Despite her academic achievements and passion for her field, Sarah struggles to gain acceptance and support for her goals.

As Sarah navigates the challenges of pursuing her dreams in a patriarchal society, she encounters skepticism and prejudice from her male peers and teachers. Her ideas are dismissed in class, and she is often overlooked for opportunities and recognition, simply because of her gender. Despite these obstacles, Sarah remains determined to prove herself and break free from societal expectations.

With the support of a few allies who recognize her potential, Sarah begins to challenge the status quo and advocate for gender equality in her community. She starts a grassroots movement to empower women and girls to pursue their ambitions, despite the limitations imposed by traditional gender roles.


"""

In [ ]:
entity_type="Misogyny"
result= llm_chain.run({"input_text":text_4,"entity_type":entity_type})
print(result)

In [ ]:
plot_3=""" Ranvijay "Vijay" Singh is the son of Balbir Singh, a powerful Delhi-based industrialist. Despite Balbir being a strict father, Vijay's love for his father is deep and abiding. Balbir doesn't know about his son's devotion for him due to his busy schedule, which also keeps him from spending time with his family.

One day, Balbir expels Vijay to a boarding school in the USA when Vijay threatens the bullies of his elder sister Reet with an AK-47. Vijay finishes his education and returns home after several years. During Balbir's birthday party, Vijay gets into an argument with his brother-in-law Varun, prompting Balbir to expel him from the house again. Vijay marries his childhood girlfriend Geetanjali from a Telugu family and the couple cuts ties with their families and moves to the USA.

Eight years later, Vijay, Geetanjali and their two kids return to India upon learning that Balbir had an assassination attempt. After meeting his father, Vijay reconciles with his family and takes it upon himself to uncover the assassin's identity. He rounds up his cousins from the village for backup and recruits a body double for his father, who is eventually killed by a gangster named Asrar Haque. Vijay discovers Varun's role in the assassination attempt and kills him publicly at a conference with his cousins, where Asrar is also present.

Vijay and his cousins meet Freddy, an arms dealer at a hotel where Asrar arrives with his assailants to kill Vijay. Vijay takes them all down with a machine gun and kills Asrar, but gets severely wounded and became bedridden with a damaged sense of hearing. He wakes up after two weeks and undergoes a heart transplant for his failing heart. Meanwhile, in Scotland, Asrar's brother Abrar Haque learns of his death during Abrar's third wedding and kills one of his men publicly at the wedding. Abrar and his other brother Abid go to a house where they shoot the man who was keeping tabs on Asrar revealing Abrar was the one who planned the assassination attempt on Balbir. Vijay recuperates after several months and is approached by Zoya, the fiancée of his heart donor. Vijay starts an extramartial affair with Zoya, who eventually reveals that she was sent by Abrar to honey trap him, so that one of his brothers who transformed himself into another Vijay, would kill his family, but Vijay reveals that he had already knew this and played along to learn the mastermind's identity.

Vijay learns from his grandfather Rajdheer Singh that Abrar and his brothers are Vijay's second cousins. They are the grandchildren of Shamsher Singh, Rajdheer's younger brother. Rajdheer had renounced Shamsher for his immoral antics. Shamsher's son Azim, who had converted to Islam and married a couple of times, was denied any share in Balbir's assets, which prompted Shamsher to commit suicide by setting himself on fire. Abrar, who had witnessed his grandfather's suicide, became mute from the trauma. Vijay then reveals to Geethanjali about the affair with Zoya and she becomes depressed and angry at Vijay but the two put aside their differences.

Vijay with his cousins stops Abrar, who is about to escape the country in a private jet with Abid. A violent fistfight ensues on the runway for a couple of hours. Vijay requests Abrar to stop targeting Balbir to which he denies. At the end of the fight Vijay brutally slits Abrar's throat, thereby putting an end to the chapter.

Back in India during Diwali, Balbir reveals that he has stage 4 ALL with only a few months left to live. He realises that his lack of love for his son has always been the root cause of Vijay's aggression. Balbir finally apologises to Vijay who bitterly cries in the pain of losing his father, the one whom he loved more than anyone. At this time, Geetanjali leaves Vijay for USA with their children, but their son Abhay runs back to Vijay.

In a post-credits scene, Aziz Haque, a butcher in Istanbul and the youngest of the Haque brothers, was the one who underwent plastic surgery to resemble another Vijay. Zoya and Abid are with him and Zoya is revealed to be pregnant, though unsure whether the child's father is Aziz or Vijay. After butchering two of Vijay's cousins brutally, Aziz sets out with Abid to exact revenge for the deaths of their two elder brothers."""

In [ ]:
ner_results = pipeline(plot_3)

# Print the entities found in the text along with their type and label
for entity in ner_results:
    print(f"Entity: {entity['word']}, Type: {entity['entity']}")